In [ ]:
import os
# tensorflow
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

data_base_path = 'C:\\Users\\User\\Downloads\\Imagens'
test_path= os.path.join(data_base_path,'test').replace(os.path.sep,"\\")
train_path=os.path.join(data_base_path,'train').replace(os.path.sep,"\\")
validation_path=os.path.join(data_base_path,'validation').replace(os.path.sep,"\\")
print(test_path)
print(train_path)
print(validation_path)

heigth,width=180,180 #config img
epochs = 20 # qnts epocas ele ira iterar para o train
batch_size = 32 # tamanho dos lotes
learnig_rate=0.0001 # taxa de aprendizagem para o optimizer

classes = ['ausente','escassa','uniforme','mediana']
count_class = len(classes)

# data set treino
dataset_treino = keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(width,heigth),
    shuffle=True,
    batch_size=batch_size,
)

# data set validacao
dataset_validacao = keras.utils.image_dataset_from_directory(
    validation_path,
    image_size=(width,heigth),
    shuffle=True,
    batch_size=batch_size,
)

# Dividindo o conjunto de validação em validação e teste
val_batches = tf.data.experimental.cardinality(dataset_validacao)
dataset_teste = dataset_validacao.take(val_batches // 2)
dataset_validacao = dataset_validacao.skip(val_batches // 2)

# Contando o número de amostras em cada conjunto
num_val_samples = sum(len(batch) for batch in dataset_validacao)
num_test_samples = sum(len(batch) for batch in dataset_teste)

print(f"Número de amostras na validação: {num_val_samples}")
print(f"Número de amostras no teste: {num_test_samples}")

from keras.layers import RandomFlip,RandomRotation,RandomZoom

# data argumentation pra evitar overfit
data_argumentation = Sequential([
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.2),
    RandomZoom((0.2,0.3))
])

# rescala
escala = layers.Rescaling(scale=1./255)

# criacao do modelo
modelo = Sequential([
    data_argumentation,
    escala,
    layers.Conv2D(32,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(256,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes,activation='softmax')
])

# copilacao do modelo 
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
  )

# resumo do modelo 
modelo.summary()

# treinamento do modelo

history = modelo.fit(
    dataset_treino, validation_data=dataset_validacao, epochs=epochs, 
)               

# Avaliação do modelo no conjunto de teste
test_loss, test_acc = modelo.evaluate(dataset_teste, verbose=2)
print("\nTest accuracy:", test_acc)

import numpy as np
from tensorflow.keras.preprocessing import image

# Função para carregar e preprocessar a imagem
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(height, width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Adiciona uma dimensão extra para o batch
    img_array = img_array / 255.0  # Normaliza a imagem como feito no treinamento
    return img_array

# Caminho para a imagem que você quer testar
img_path = 'C:\\Users\\User\\Downloads\\Imagens\\test_image.jpg'

# Preprocessa a imagem
img_array = preprocess_image(img_path)

# Faz a predição
predictions = modelo.predict(img_array)

# Obtém a classe prevista
predicted_class = np.argmax(predictions, axis=1)
predicted_class_label = classes[predicted_class[0]]

print(f"Predição: {predicted_class_label}")